[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/eneskelestemur/MolecularModeling/blob/main/Labs/lab08_molecular_dynamics/Molecular_dynamics.ipynb)

In this exercise, we'll use `openmm`, `openforcefields`, `openff-toolkit` and `mdanalysis` packages. Their recommended way of installation is through Ancaonda/Miniconda, so before starting this notebook prepare the environment using Anaconda. 

If running in Google Colab, run the following code first.

In [ ]:
# Only for Google Colab
!pip install -q condacolab
import condacolab
condacolab.install()

In [ ]:
# Only for Google Colab
!mamba install -c conda-forge openmm openmmforcefileds openff-toolkit pdbfixer mdanalysis nglview pandas matplotlib -y

# Molecular Dynamics Using Openmm

In this lab, we will demonstrate Molecular Dynamics Simulation and its analysis in Python.

* [Estrogen](#estrogen)
* [Estrogen-Raloxifene](#estrogen-raloxifene)
* [Ras-Raf](#ras-raf)

## Estrogen

The first example will be a simple protein only system. We will simulate Estrogen protein for a short amount of time analyze the simulation run afterwards.

In [ ]:
# Let's visualize the protein using NGLView
import nglview as nv
import MDAnalysis as mda

# Load the protein structure
estrogen = 'data/Estrogen_Receptor.pdb'
u = mda.Universe(estrogen)
nv.show_mdanalysis(u, gui=True)

### Basic MD Setup

In [ ]:
import openmm.app as app
import openmm as mm
from openmm import unit
from sys import stdout

# Set the path to the PDB file
estrogen_file = 'data/Estrogen_Receptor.pdb'
traj_output_file = 'data/Estrogen_Receptor.pdb'
state_output_file = 'data/Estrogen_Receptor.log'

# 1.loading initial coordinates
pdb = app.PDBFile(estrogen_file)

# 2.choosing a forcefield parameters
ff = app.ForceField('amber14-all.xml')
system = ff.createSystem(pdb.topology, nonbondedMethod=app.CutoffNonPeriodic)

# 3. Choose parameters of the experiment: temperature, pressure, box size, solvation, boundary conditions, etc
temperature = 300*unit.kelvin
frictionCoeff = 1/unit.picosecond
time_step = 0.002*unit.picoseconds
total_steps = 400*unit.picoseconds / time_step

# 4. Choose an algorithm (integrator)
integrator = mm.LangevinIntegrator(temperature, frictionCoeff, time_step)

# 5. Run simulation, saving coordinates time to time:

# 5a. Create a simulation object
simulation = app.Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)

# 5b. Minimize energy
simulation.minimizeEnergy()

# 5c. Save coordinates to pdb file and energies to a standard output console:
simulation.reporters.append(app.PDBReporter(traj_output_file, 1000))
simulation.reporters.append(app.StateDataReporter(state_output_file, 5000, step=True, potentialEnergy=True,
                                                  temperature=True, progress=True, totalSteps = total_steps))

# 5d. Run!
simulation.step(total_steps)


In [ ]:
# 6. Visualization
u = mda.Universe(estrogen_file, traj_output_file)
nv.show_mdanalysis(u, gui=True)

### Analyzing the Simulation

In [ ]:
import MDAnalysis as mda
import pandas as pd
import matplotlib.pyplot as plt

# Plot state data
def plot_StateData(data_file, names_to_plot, 
                   sim_step_size=0.002, 
                   report_interval=1000):
    n_names = len(names_to_plot)
    data = pd.read_csv(data_file, index_col=None)
    try:
        sim_time = data['Step'] * sim_step_size / report_interval # ns
    except KeyError:
        sim_time = data['#"Step"'] * sim_step_size / report_interval # ns
    fig = plt.figure(figsize=(20, 10))
    for i, name in enumerate(names_to_plot):
        ax = fig.add_subplot((n_names+1)//2, 2, i+1)
        ax.plot(sim_time, data[name])
        ax.set_xlabel('Time (ns)')
        ax.set_ylabel(name)
    fig.tight_layout()
    plt.show()

plot_StateData(
    state_output_file,
    ['Potential Energy (kJ/mole)', 'Total Energy (kJ/mole)', 'Temperature (K)', 'Density (g/mL)']
)

In [ ]:
from MDAnalysis.analysis import rms, align

## RMSD calculation
# calculate rmsd
R = rms.RMSD(u,  # universe to align
             u,  # reference universe or atomgroup
             select='backbone',  # group to superimpose and calculate RMSD
             groupselections=['segid A'],  # groups for RMSD
             ref_frame=0)  # frame index of the reference
R.run()

## RMSF Analysis
# create the average structure of the trajectory
average = align.AverageStructure(u, u, select='protein and name CA',
                                 ref_frame=0).run()
ref = average.results.universe
# Align the trajectory to the reference
aligner = align.AlignTraj(u, ref,
                          select='protein and name CA',
                          in_memory=True).run()

# Calculate RMSF
c_alphas = u.select_atoms('protein and name CA')
F = rms.RMSF(c_alphas).run()

In [ ]:
# plot the RMSD for the protein
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(121)
ax.plot(R.results.rmsd[:,0], R.results.rmsd[:,2], label="Backbone")
ax.set_xlabel("Frame")
ax.set_ylabel(r'RMSD ($\AA$)')
ax.legend()

# plot RMSF for the protein
ax = fig.add_subplot(122)
ax.plot(c_alphas.resids, F.results.rmsf)
ax.set_xlabel("Residue Number")
ax.set_ylabel("RMSF ($\AA$)")
ax.legend()
fig.show()

In [ ]:
from MDAnalysis.analysis import diffusionmap

## Plot pairwise RMSD
# load the trajectory
u = mda.Universe(estrogen_file, traj_output_file, all_coordinates=True)

# align the trajectory to the first frame
align.AlignTraj(u, u, select='backbone', in_memory=True).run()

# calculate the pairwise RMSD
matrix = diffusionmap.DistanceMatrix(u, select='all').run()
res = matrix.results.dist_matrix

# plot the pairwise RMSD
fig = plt.figure(figsize=(12, 9), dpi=150)
ax = fig.add_subplot(111)
ax.imshow(res, cmap='viridis')
ax.set_xlabel('Frame')
ax.set_ylabel('Frame')
fig.colorbar(mappable=ax.imshow(res, cmap='viridis'), label='RMSD (Å)')
fig.tight_layout()
fig.show()